# Generate Music Lyrics

Dataset: https://www.kaggle.com/mousehead/songlyrics

## Import libs

In [1]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
!pip install unidecode

## Get dataset

In [0]:
# comente linhas seguinte caso rode mais de uma vez
!wget https://www.dropbox.com/s/gspbmxiabhwrce3/english_lyrics.txt?dl=0
############################################################

In [0]:
import unidecode
import string
import random
import re

all_characters = string.printable
n_characters = len(all_characters)
chunk_len = 200

file = open('english_lyrics.txt?dl=0').read()
file_len = len(file)

# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)
  
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]
  
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

## Set architecture

Camadas:
* Embedding: https://pytorch.org/docs/stable/nn.html#torch.nn.Embedding
* GRU: https://pytorch.org/docs/stable/nn.html#torch.nn.GRU
* Linear: https://pytorch.org/docs/stable/nn.html#torch.nn.Linear

Auxiliares:
View: https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True, dropout=0.5)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden, predict=0):
        
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input, hidden)
        output = self.decoder(output.view(-1, output.size(-1)))
        
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size)).cuda()

## Train / evaluate model

In [0]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state

    _, hidden = decoder(prime_input.cuda(), hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp.cuda(), hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

In [0]:
def train(inp, target):
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    loss = 0

    output, hidden = decoder(inp.cuda(), hidden)
    loss += criterion(output, target.cuda())

    loss.backward()
    decoder_optimizer.step()

    return loss.data[0] / chunk_len

In [7]:
n_steps = 2000
print_every = 100
plot_every = 100

embedding_size = 128
hidden_size = 128
n_layers = 2
lr = 0.005

print('Start')
decoder = RNN(n_characters, embedding_size, hidden_size, n_characters, n_layers)
decoder.cuda()
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss().cuda()

all_losses = []
loss_avg = 0

print('Before Training...')
print(evaluate('I', 100), '\n')

for step in range(1, n_steps + 1):
    decoder.train()
    loss = train(*random_training_set())       
    loss_avg += loss

    if step % print_every == 0:
        decoder.eval()
        print('[(%d %d%%) %.4f]' % (step, step / n_steps * 100, loss))
        print(evaluate('I', 100), '\n')

    if step % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

Start
[(100 5%) 0.0107]
I epeng teeee detelrer at  
I'm lerte min thele letete young I an thehe 3ee eol you foteat cene peat  

[(200 10%) 0.0105]
I's no you do the love to mew mat it  
Whet's-pover I  
To we maist you wen whan forind doll  
Gow in 

[(300 15%) 0.0130]
I done hainre  
Sid my you houc whin' that know low cander tho wus the  
And ending staning I by boon 

[(400 20%) 0.0105]
I on to fore you siin hare in the stat the  
There  
Und I we of you what oned  
Needs the a ke hatty 

[(500 25%) 0.0102]
I)  
And the whers know feel a dorling  
Dot's wnow  
List like to roch day  
Kink I't the  
All be l 

[(600 30%) 0.0126]
I cand I see  
  
Chere is cap  
  
Leatk  
Bomes a hite  
  
  
Pall gong  
I wake  
The felle on sa 

[(700 35%) 0.0093]
I it  
To festing reall  
Gas  
I feat  
I'm nay agay  
I'm love I knone  
  
For it I'm beel love re 

[(800 40%) 0.0083]
I sime  
One you go ging you got  
Nour your a pirth and it shooting  
So sary as thing in hive a fie 

[(900 45%) 